In [1]:
import sys
import os
sys.path.append(os.path.abspath("Pytorch/"))
sys.path.append(os.path.abspath("models/"))

In [2]:
from XVA import *
from XVAFBSNNs import *

In [3]:
from FBSNNs import *
from CallOptionsBasket import *

In [4]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 200  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [D + 1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "NAIS-Net"  # FC, Resnet and NAIS-Net are available
activation = "Sine"  # Sine and ReLU are available
model = CallOptionsBasket(Xi, T, M, N, D, Mm, layers, mode, activation)

n_iter = 2 * 10**4
lr = 1e-3

In [5]:
model.load_model("models/100DCallOptionXVAPaper.pth")

In [6]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 200  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [1+1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "NAIS-Net"  # FC, Resnet and NAIS-Net are available
activation = "Sine"  # Sine and ReLU are available
xvamodel = XVA(Xi, T, M, N, D, Mm, layers, mode, activation, model)

n_iter = 2 * 10**3
lr = 1e-3

In [7]:
xvamodel.load_model("models/100DCallOptionFVAXVAPaper.pth")

In [10]:
n_iter = 1 * 10**3
lr = 1e-5

In [11]:
tot = time.time()
print(xvamodel.device)
graph = xvamodel.train(n_iter, lr)
print("total time:", time.time() - tot, "s")

cuda:0
It: 3700, Loss: 1.618e-04, Y0: 0.044, Time: 0.90, Learning Rate: 1.000e-05
total time: 5.886070966720581 s


In [8]:
t_test, W_test, C_test = xvamodel.fetch_minibatch()
X_pred, Y_pred = xvamodel.predict(C_test, t_test, W_test)

if type(t_test).__module__ != 'numpy':
    t_test = t_test.cpu().numpy()
if type(X_pred).__module__ != 'numpy':
    X_pred = X_pred.cpu().detach().numpy()
if type(Y_pred).__module__ != 'numpy':
    Y_pred = Y_pred.cpu().detach().numpy()

In [9]:
Y_pred.shape

(1, 201, 1)

In [11]:
Y_preds = Y_pred.reshape(1, 201)

In [12]:
pred_fva = Y_preds[:,0][0]

In [13]:
from scipy.stats import multivariate_normal as normal
np.random.seed(0)
dw_sample = normal.rvs(size=[50000, D, N]) * np.sqrt(T/N)
x_sample = np.zeros([50000, D, N + 1]) 
x_sample[:, :, 0] = np.ones([50000, D]) * 1
factor = np.exp((0.01-(0.25**2)/2)*(T/N))
for i in range(N):   
    x_sample[:, :, i + 1] = (factor * np.exp(0.25 * dw_sample[:, :, i])) * x_sample[:, :, i]
np.random.seed(1)
dw_sample1 = normal.rvs(size=[50000, D, N]) * np.sqrt(T/N)
x_sample1 = np.zeros([50000, D, N + 1]) 
x_sample1[:, :, 0] = np.ones([50000, D]) 
factor = np.exp((0.01-(0.25**2)/2)*(T/N))
for i in range(N):   
    x_sample1[:, :, i + 1] = (factor * np.exp(0.25 * dw_sample1[:, :, i])) * x_sample1[:, :, i]

In [14]:
portfolio1 = np.sum(x_sample, axis=1, keepdims=True)
portfolio2 = np.sum(x_sample1, axis=1, keepdims=True)

In [15]:
payoff1 = np.maximum(portfolio1 - 100, 0)
payoff2 = np.maximum(portfolio2 - 100, 0)

In [16]:
average_1 = np.mean(payoff1, axis=0, keepdims=True)
average_2 = np.mean(payoff2, axis=0, keepdims=True)

In [17]:
average = (average_1 + average_2)/ 2

In [18]:
mc_price = np.exp(-0.01 * 1) * average[:, :, 200]

In [19]:
mc_price[0][0]

1.5799232739802955

In [22]:
mc_price_rf = np.exp(-0.04 * 1) * average[:, :, 200]

In [23]:
mc_price_rf[0][0]

1.5332294845835137

In [24]:
exact_fva = mc_price[0][0] - mc_price_rf[0][0]

In [25]:
pred_fva * 100

4.6674929559230804

In [26]:
exact_fva * 100

4.6693789396781815

In [27]:
(pred_fva - exact_fva) ** 2 / exact_fva ** 2

1.6313894704572896e-07

In [29]:
np.abs(pred_fva - exact_fva) * 100

0.0018859837551010727

In [36]:
xvamodel.save_model("models/100DCallOptionFVAXVAPaper.pth")